In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (26.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [ ]:
!mkdir fonts output train

In [ ]:
font_name = 'Seven Segment' #@param {type: "string"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!chmod 755 -R /content/drive/MyDrive/tesseract/src/training/tesstrain.sh

In [ ]:
!combine_tessdata -e /content/drive/MyDrive/tesseract/tessdata/eng.traineddata eng.lstm

Extracting tessdata components from /content/drive/MyDrive/tesseract/tessdata/eng.traineddata
Wrote eng.lstm
Version string:4.00.00alpha:eng:synth20170629:[1,36,0,1Ct3,3,16Mp3,3Lfys64Lfx96Lrx96Lfx512O1c1]
17:lstm:size=11689099, offset=192
18:lstm-punc-dawg:size=4322, offset=11689291
19:lstm-word-dawg:size=3694794, offset=11693613
20:lstm-number-dawg:size=4738, offset=15388407
21:lstm-unicharset:size=6360, offset=15393145
22:lstm-recoder:size=1012, offset=15399505
23:version:size=80, offset=15400517


In [ ]:
!rm -rf train/*
! /content/drive/MyDrive/tesseract/src/training/tesstrain.sh --fonts_dir fonts \
	--fontlist '{font_name}' \
	--lang eng \
	--linedata_only \
	--langdata_dir /content/drive/MyDrive/langdata_lstm \
	--tessdata_dir /content/drive/MyDrive/tesseract/tessdata \
	--save_box_tiff \
	--maxpages 10 \
	--output_dir train


=== Starting training for language 'eng'
[Wed May 15 08:49:15 AM UTC 2024] /usr/bin/text2image --fonts_dir=fonts --ptsize 12 --font=Seven Segment --outputbase=/tmp/font_tmp.e3Tv0N53cR/sample_text.txt --text=/tmp/font_tmp.e3Tv0N53cR/sample_text.txt --fontconfig_tmpdir=/tmp/font_tmp.e3Tv0N53cR
Rendered page 0 to file /tmp/font_tmp.e3Tv0N53cR/sample_text.txt.tif

=== Phase I: Generating training images ===
Rendering using Seven Segment
[Wed May 15 08:49:17 AM UTC 2024] /usr/bin/text2image --fontconfig_tmpdir=/tmp/font_tmp.e3Tv0N53cR --fonts_dir=fonts --strip_unrenderable_words --leading=32 --xsize=3600 --char_spacing=0.0 --exposure=0 --outputbase=/tmp/eng-2024-05-15.srG/eng.Seven_Segment.exp0 --max_pages=10 --font=Seven Segment --ptsize 12 --text=/content/drive/MyDrive/langdata_lstm/eng/eng.training_text
Stripped 50 unrenderable words
Rendered page 0 to file /tmp/eng-2024-05-15.srG/eng.Seven_Segment.exp0.tif
Stripped 37 unrenderable words
Rendered page 1 to file /tmp/eng-2024-05-15.srG/e

In [ ]:
%%shell

rm -rf output/*
OMP_THREAD_LIMIT=16 lstmtraining \
	--continue_from eng.lstm \
	--model_output output/font_name \
	--traineddata /content/drive/MyDrive/tesseract/tessdata/eng.traineddata \
	--train_listfile train/eng.training_files.txt \
	--max_iterations 200

Loaded file eng.lstm, unpacking...
Continuing from eng.lstm
Loaded 540/540 lines (1-540) of document train/eng.Seven_Segment.exp0.lstmf
2 Percent improvement time=100, best error was 100 @ 0
At iteration 100/100/100, Mean rms=4.004%, delta=23.856%, char train=70.103%, word train=85.017%, skip ratio=0%,  New best char error = 70.103 wrote best model:output/font_name70.103_100.checkpoint wrote checkpoint.

2 Percent improvement time=100, best error was 70.103 @ 100
At iteration 200/200/200, Mean rms=3.775%, delta=21.782%, char train=65.023%, word train=85.908%, skip ratio=0%,  New best char error = 65.023 wrote best model:output/font_name65.023_200.checkpoint wrote checkpoint.

Finished! Error rate = 65.023


In [ ]:
%%shell

lstmtraining --stop_training \
	--continue_from output/font_name_checkpoint \
	--traineddata /content/drive/MyDrive/tesseract/tessdata/eng.traineddata \
	--model_output output/font_name.traineddata

Loaded file output/font_name_checkpoint, unpacking...
